In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

import random
import pandas as pd
from datetime import datetime
import re
import time

In [ ]:
delay = random.uniform(1, 3)
delay_short = random.uniform(0.5, 1)

# RECORD LIST OF LOCATION
results = []

list_target = [
    {
        "kota": "Jakarta Selatan",
        "kecamatan": "Cilandak",
        "lat": "-6.284528",
        "long": "106.800140",
    },
]

def genarate_lat_long(url_detail): 
    match = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', url_detail)
    latitude = ""
    longitude = ""

    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))

    return{
        "latitude": latitude,
        "longitude": longitude
    }

def get_element_text(navigator, selector): 
    try:
        element = wait.until(EC.visibility_of_element_located((navigator, selector)))
        return element.text 
    except StaleElementReferenceException:
        return get_element_text(navigator, selector)
    except Exception as error:
        print(f"An exception was thrown!:,{error}")
        return "" 

for index, target in enumerate(list_target): 
    print(f"target : {target}") 
    
    #  Web Driver
    options = uc.ChromeOptions()
    driver = uc.Chrome(options=options)
    wait = WebDriverWait(driver, delay_short)

    # PAGE PARAMETERS
    keyword = "toko+kelontong" 
    zoom = 16

    # OPEN PAGE
    url = f"https://www.google.com/maps/search/{keyword}/@{target['lat']},{target['long']},{zoom}z/data=!3m1!4b1?entry=ttu"
    driver.get(url) 
    time.sleep(delay) 

    scrollable_list = driver.find_element(By.XPATH, "//div[contains(@role,'feed')]")
    max_scrolls = 100  # Maximum number of scrolls to perform
    # max_scrolls = 3  # Maximum number of scrolls to perform
    scroll_count = 0
    end_condition_met = False
    while not end_condition_met and scroll_count < max_scrolls:
        try:
            driver.find_element(By.CSS_SELECTOR, '.HlvSq').text
            end_condition_met = True
            print("End of Page")
        except NoSuchElementException:
            scrollable_list.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)  # Adjust this time as needed
            scroll_count += 1
            print(f"scroll_count: {scroll_count}")
    print("-" * 50)
  
    location_list = driver.find_elements(By.CSS_SELECTOR, '.hfpxzc') # list of location  
    # location_list = driver.find_elements(By.XPATH, "//a[contains(@class,'hfpxzc')]")

    for index, location in enumerate(location_list):
        # if index > 10:
        #     break
        
        # location.click() 
 
        try:
            location.click()
        except Exception as error:
            print(f"An exception was thrown!:,{error}")
            continue 

        title = get_element_text(By.CSS_SELECTOR, '.DUwDvf.lfPIob' )
        phone = get_element_text(By.XPATH, "//button[contains(@data-item-id,'phone')]")
        address = get_element_text(By.CSS_SELECTOR, '.Io6YTe.fontBodyMedium.kR99db')
        rating = get_element_text(By.CSS_SELECTOR, '.fontDisplayLarge')
        reviewer = get_element_text(By.CSS_SELECTOR, '.HHrUdb.fontTitleSmall.rqjGif')
        location_detail_url = location.get_attribute('href') or ""
        coordinat = genarate_lat_long(location.get_attribute('href')) 

        result = {
            "title" : title,
            "city": target["kota"],
            "Kecamatan": target["kecamatan"],
            "phone" : phone,
            "address" : address,
            "rating" : rating,
            "reviewer" : reviewer,
            "url" : location_detail_url,
            "latitude" : coordinat["latitude"],
            "longitude" : coordinat["longitude"],
        }
        results.append(result)
    driver.quit()

dt_now = datetime.now().strftime('%Y%m%d_%H_%M_%S')
df_data = pd.DataFrame(results)
df_data.to_csv(f'{keyword}{dt_now}.csv',index=False)
print("---DONE--")
print(results)